# Import

In [1]:
from qDrift.hamsimqDrift import AlgorithmHamSimqDrift
from pytket.circuit import Circuit
from openfermion.utils.operator_utils import count_qubits

from utils.func import *
from utils.term_grouping import *
from utils.min_clique import *
from utils.hamlib import *
from utils.evol_real import *
from utils.plot import *
from utils.driver import *
from utils.JW import *

from Pauli_Gadgets.paulis import *

import numpy as np
import matplotlib.pyplot as plt

def gen_sys_param(n=10, sample_steps=100000, N=1):
    global t_max
    global initial_state_circ
    global n_qdrift_steps
    global sample_space
    global seg
    global t_list

    t_list = [0.5,1,2,5]
    t_max=t_list[-1]
    seg = N
    initial_state_circ = Circuit(n)
    for i in range(n):
        initial_state_circ.H(i)
    n_qdrift_steps = sample_steps
    sample_space = np.arange(0,sample_steps+1,1)

# Hydrogen chain

In [2]:
driver = PySCFDriver_custom()
bond_length = 1.0
n_sites = 3
driver.load_molecule(
    atom=[('H', (0, 0, i * bond_length)) for i in range(n_sites)], # Create a molecular data object for the hydrogen chain
    basis="sto-3g",          # Basis set for quantum chemistry calculation
    multiplicity=1,          # Spin multiplicity for molecule, since the total spin of H2O is S=0，its spin multiplicity is 2S+1=1
    charge=0,                 # Total charge of molecule, since H2O is charge neutral, its charge=0
    unit="Angstrom"
)
driver.run_scf()             # Perform Hartree Fock calculation

# np.set_printoptions(precision=4, linewidth=150)

hpq = driver.get_onebody_tensor("int1e_kin") + driver.get_onebody_tensor("int1e_nuc")
vpqrs = driver.get_twobody_tensor()
# assert np.shape(hpq)==(7, 7)             # H2O has 7 orbitals when using STO-3G basis.
# assert np.shape(vpqrs)==(7, 7, 7, 7)

# print(hpq)
operator = get_molecular_hamiltonian(hpq,vpqrs,driver)
n_qubits = count_qubits(operator)
number, coulomb, hopping, no_excitation, double_excitation = JW_transformation(operator)
numbers, coulombs, hoppings, no_excitations, double_excitations = JW_transformation(operator,True)

number_op, number_co = convert_op_to_input(number,n_qubits)
hopping_op, hopping_co = convert_op_to_input(hopping,n_qubits)
coulomb_op, coulomb_co = convert_op_to_input(coulomb,n_qubits)
no_excitation_op, no_excitation_co = convert_op_to_input(no_excitation,n_qubits)
double_excitation_op, double_excitation_co = convert_op_to_input(double_excitation,n_qubits)

numbers_op, numbers_co = convert_op_to_input(numbers,n_qubits,True)
hoppings_op, hoppings_co = convert_twobody_op_to_input(hoppings,n_qubits,True)
coulombs_op, coulombs_co = convert_op_to_input(list(coulombs),n_qubits,True)
no_excitations_op, no_excitations_co = convert_twobody_op_to_input(list(no_excitations),n_qubits,True)
double_excitations_op, double_excitations_co = convert_twobody_op_to_input(list(double_excitations),n_qubits,True)

# max_part_group = [[number_op[0], number_op[-2],hopping_op[0], hopping_op[1]],number_op[2:4],[number_op[1], number_op[-1], hopping_op[-2],hopping_op[-1]]]+coulombs_op+no_excitations_op+double_excitations_op
# max_coeff = [[(number_co[0]+number_co[-2]+abs(hopping_co[0])*2)/4],[number_co[2]],[(number_co[0]+number_co[-2]+abs(hopping_co[0])*2)/4]]+coulombs_co+no_excitations_co+double_excitations_co
# max_part_group, max_coeff = site_excitation_group(numbers_op,hoppings_op,numbers_co,hoppings_co)

H_matrix = get_Hmatrix(number_op+hopping_op+coulomb_op+no_excitation_op+double_excitation_op,n_qubits,number_co+hopping_co+coulomb_co+no_excitation_co+double_excitation_co)

converged SCF energy = -1.52399620024611


In [3]:
# gen_sys_param(n=n_qubits,sample_steps=1000)
# drift_time_evolution = AlgorithmHamSimqDrift(initial_state_circ,number_op+hopping_op+coulomb_op+no_excitation_op+double_excitation_op,number_co+hopping_co+coulomb_co+no_excitation_co+double_excitation_co,t_max,n_qdrift_steps,seg,noise=True)
# U2p,secdepth = drift_time_evolution.trotter(order=2, protected=True,cheat=True)

In [4]:
# gen_sys_param(n=n_qubits,sample_steps=2000)
# drift_time_evolution = AlgorithmHamSimqDrift(initial_state_circ,number_op+hopping_op+coulomb_op+no_excitation_op+double_excitation_op,number_co+hopping_co+coulomb_co+no_excitation_co+double_excitation_co,t_max,n_qdrift_steps,seg,noise=True)
# Ut,ldepth = drift_time_evolution.trotter(cheat=True)

In [5]:
# gen_sys_param(n=n_qubits,sample_steps=2000)
# drift_time_evolution = AlgorithmHamSimqDrift(initial_state_circ,number_op+hopping_op+coulomb_op+no_excitation_op+double_excitation_op,number_co+hopping_co+coulomb_co+no_excitation_co+double_excitation_co,t_max,n_qdrift_steps,seg,noise=True)
# Utp,ldepth = drift_time_evolution.trotter(protected=True,cheat=True)

In [6]:
gen_sys_param(n=n_qubits,sample_steps=1000)
drift_time_evolution = AlgorithmHamSimqDrift(initial_state_circ,number_op+hopping_op+coulomb_op+no_excitation_op+double_excitation_op,number_co+hopping_co+coulomb_co+no_excitation_co+double_excitation_co,t_max,n_qdrift_steps,seg,noise=True)
U2,secdepth = drift_time_evolution.trotter(order=2,cheat=True)

In [7]:
# Urp_sum = []
# for i in range(3):
#     gen_sys_param(n=n_qubits,sample_steps=2000)
#     drift_time_evolution = AlgorithmHamSimqDrift(initial_state_circ,number_op+hopping_op+coulomb_op+no_excitation_op+double_excitation_op,number_co+hopping_co+coulomb_co+no_excitation_co+double_excitation_co,t_max,n_qdrift_steps,seg)
#     Urp_i,rpdepth = drift_time_evolution.trotter(rand=True, cheat=True)
#     Urp_sum.append(Urp_i)
# Urp = [np.add(np.add(np.array(Urp_sum[0][i]),np.array(Urp_sum[1][i])),np.array(Urp_sum[2][i]))/3 for i in range(len(Urp_sum[0]))]

In [8]:
gen_sys_param(n=n_qubits,sample_steps=secdepth[-1])
drift_time_evolution = AlgorithmHamSimqDrift(initial_state_circ,numbers_op+hoppings_op+coulombs_op+no_excitations_op+double_excitations_op,numbers_co+hoppings_co+coulombs_co+no_excitations_co+double_excitations_co,t_max,n_qdrift_steps,seg,M=3,noise=True)
Um,sm,mdepth = drift_time_evolution.Drift_exp(depth=secdepth[-1], cheat=True)
Um= [[Um[j][0][i] for i in range(len(Um[j][0]))] for j in range(len(Um))]
mdepth = [[mdepth[j][0][i] for i in range(len(mdepth[j][0]))] for j in range(len(mdepth))]

In [9]:
# Vmp, coeffmp = perm_ops(t_list,mdepth,sm)

In [10]:
# gen_sys_param(n=n_qubits,sample_steps=secdepth[-1])
# drift_time_evolution = AlgorithmHamSimqDrift(initial_state_circ,numbers_op+hoppings_op+coulombs_op+no_excitations_op+double_excitations_op,numbers_co+hoppings_co+coulombs_co+no_excitations_co+double_excitations_co,t_max,n_qdrift_steps,seg,M=3,noise=True)
# Ump,smp,mdepthp = drift_time_evolution.Drift_exp(sampled=[Vmp,coeffmp,sm[-1]],depth=secdepth[-1], cheat=True)
# Ump= [[Ump[j][0][i] for i in range(len(Ump[j][0]))] for j in range(len(Ump))]
# mdepthp = [[mdepthp[j][0][i] for i in range(len(mdepthp[j][0]))] for j in range(len(mdepthp))]

In [11]:
gen_sys_param(n=n_qubits,sample_steps=secdepth[-1])
drift_time_evolution_parity = AlgorithmHamSimqDrift(initial_state_circ,[[h] for h in number_op+hopping_op+coulomb_op+no_excitation_op+double_excitation_op],[[c] for c in number_co+hopping_co+coulomb_co+no_excitation_co+double_excitation_co],t_max,n_qdrift_steps,seg,M=3)
Uq,sq,qdepth = drift_time_evolution_parity.Drift_exp(depth=secdepth[-1], cheat=True)
Uq= [[Uq[j][0][i] for i in range(len(Uq[j][0]))] for j in range(len(Uq))]
qdepth = [[qdepth[j][0][i] for i in range(len(qdepth[j][0]))] for j in range(len(qdepth))]

In [12]:
# Vqp, coeffqp = perm_ops(t_list,qdepth,sq)

In [13]:
# gen_sys_param(n=n_qubits,sample_steps=secdepth[-1])
# drift_time_evolution_parity = AlgorithmHamSimqDrift(initial_state_circ,[[h] for h in number_op+hopping_op+coulomb_op+no_excitation_op+double_excitation_op],[[c] for c in number_co+hopping_co+coulomb_co+no_excitation_co+double_excitation_co],t_max,n_qdrift_steps,seg,M=3,noise=True)
# Uqp,sqp,qdepthp = drift_time_evolution_parity.Drift_exp(sampled=[Vqp,coeffqp,sq[-1]],depth=secdepth[-1], cheat=True)
# Uqp= [[Uqp[j][0][i] for i in range(len(Uqp[j][0]))] for j in range(len(Uqp))]
# qdepthp = [[qdepthp[j][0][i] for i in range(len(qdepthp[j][0]))] for j in range(len(qdepthp))]

In [14]:
gen_sys_param(n=n_qubits,sample_steps=secdepth[-1])
Uexc = U_exc(drift_time_evolution.circuit.get_unitary(),n_qdrift_steps,t_max, H_matrix)

In [15]:
# u2p = extract_U_at_t(t_list,U2p,secdepth)
# ut = extract_U_at_t(t_list,Ut,ldepth)
# utp = extract_U_at_t(t_list,Utp,ldepth)
# u2 = extract_U_at_t(t_list,U2,secdepth)
# urp = extract_U_at_t(t_list,Urp,rpdepth)
# # um_ave = extract_U_at_t(t_list,ave_m,ave_mdepth)
# # uq_ave = extract_U_at_t(t_list,ave_q,ave_qdepth)
uexc = extract_U_at_t(t_list,Uexc,secdepth)
# m_mean,m_std = Monte_Carlo_ave(t_list,Um,mdepth,uexc,M=3)
# # g_mean,g_std = Monte_Carlo_ave(t_list,Ug,gdepth,uexc,M=3)
# q_mean,q_std = Monte_Carlo_ave(t_list,Uq,qdepth,uexc,M=3)

In [16]:
# um_spec_ave = cal_std_drift(Um, Uexc, t_list, mdepth, secdepth,True)
# uq_spec_ave = cal_std_drift(Uq, Uexc, t_list, qdepth, secdepth, True)
# ump_spec_ave = cal_std_drift(Ump, Uexc, t_list, mdepthp, secdepth,True)
# uqp_spec_ave = cal_std_drift(Uqp, Uexc, t_list, qdepthp, secdepth, True)

In [17]:
# um_spec_mean, um_spec_std = cal_std_drift(Um, Uexc, t_list, mdepth, secdepth)
# uq_spec_mean, uq_spec_std = cal_std_drift(Uq, Uexc, t_list, qdepth, secdepth)
# ump_spec_mean, ump_spec_std = cal_std_drift(Ump, Uexc, t_list, mdepthp, secdepth)
# uqp_spec_mean, uqp_spec_std = cal_std_drift(Uqp, Uexc, t_list, qdepthp, secdepth)

In [18]:
um = [extract_U_at_t(t_list,Um[i],secdepth) for i in range(len(Um))]
um_spec = [[np.abs(linalg.eig(u - u_exc)[0]).max() for u,u_exc in zip(um[i],uexc)] for i in range(len(um))]
uq = [extract_U_at_t(t_list,Uq[i],secdepth) for i in range(len(Uq))]
uq_spec = [[np.abs(linalg.eig(u - u_exc)[0]).max() for u,u_exc in zip(uq[i],uexc)] for i in range(len(uq))]

# ump = [extract_U_at_t(t_list,Ump[i],secdepth) for i in range(len(Ump))]
# ump_spec = [[np.abs(linalg.eig(u - u_exc)[0]).max() for u,u_exc in zip(ump[i],uexc)] for i in range(len(ump))]
# uqp = [extract_U_at_t(t_list,Uqp[i],secdepth) for i in range(len(Uqp))]
# uqp_spec = [[np.abs(linalg.eig(u - u_exc)[0]).max() for u,u_exc in zip(uqp[i],uexc)] for i in range(len(uqp))]

In [25]:
print(np.mean(um_spec,axis=0)[2],np.mean(uq_spec,axis=0)[2])

0.0034735604786311682 0.004657640412824799


In [20]:
# fig, ax = plt.subplots()
# ts = [0]+t_list
# ax.errorbar([0]+t_list,um_spec_ave, xerr=extract_depth_at_t(t_list,mdepth), yerr=ump_spec_std, label='physDrift',c='r')
# ax.errorbar([0]+t_list,uq_spec_ave, xerr=extract_depth_at_t(t_list,qdepth), yerr=uqp_spec_std, label='qDrift',c='b')
# ax.set_xlabel(r'$t_{ma}x}(s)$')
# ax.set_ylabel('Spectral error')
# ax.set_title(r'Exact spectral error over different $t_{max}$', pad=15) 
# ax2 = ax.secondary_xaxis('top')
# ax2.set_xlabel('No. Pauli Gadgets')
# ax2.tick_params(axis='x', color='black')
# ax2.set_xticks(ts, labels=[str(int(t/t_list[-1]*secdepth[-1])) for t in ts], minor=False)
# plt.legend()
# plt.show()

In [21]:
# fig, ax = plt.subplots()
# ts = [0]+t_list
# ax.errorbar([0]+t_list,ump_spec_ave, xerr=extract_depth_at_t(t_list,mdepth), label='physDrift',c='r')
# ax.errorbar([0]+t_list,uqp_spec_ave, xerr=extract_depth_at_t(t_list,qdepth),label='qDrift',c='b')
# ax.set_xlabel(r'$t_{ma}x}(s)$')
# ax.set_ylabel('Spectral error')
# ax.set_title(r'Exact spectral error over different $t_{max}$ with perm', pad=15) 
# ax2 = ax.secondary_xaxis('top')
# ax2.set_xlabel('No. Pauli Gadgets')
# ax2.tick_params(axis='x', color='black')
# ax2.set_xticks(ts, labels=[str(int(t/t_list[-1]*secdepth[-1])) for t in ts], minor=False)
# plt.legend()
# plt.show()

In [22]:
# fig, ax = plt.subplots()
# ts = [0]+t_list
# # ax.errorbar([0]+t_list,um_spec_mean, xerr=extract_depth_at_t(t_list,mdepth),yerr=um_spec_std, label='physDrift',c='r')
# # ax.errorbar([0]+t_list,uq_spec_mean, xerr=extract_depth_at_t(t_list,qdepth),yerr=uq_spec_std, label='qDrift',c='b')
# ax.errorbar([0]+t_list,np.mean(um_spec,axis=0), xerr=extract_depth_at_t(t_list,mdepth),yerr=np.std(um_spec,axis=0), label='physDrift',c='r')
# ax.errorbar([0]+t_list,np.mean(uq_spec,axis=0), xerr=extract_depth_at_t(t_list,qdepth),yerr=np.std(uq_spec,axis=0), label='qDrift',c='b')
# for i in range(len(um_spec)):
#     ax.scatter([0]+t_list,um_spec[i],marker='x',s=5,color='r')
# for i in range(len(uq_spec)):
#     ax.scatter([0]+t_list,uq_spec[i],marker='x',s=5,color='b')
# ax.set_xlabel(r'$t_{max}(s)$')
# ax.set_ylabel(r'Spectral error')
# ax.set_title(r'Spectral error over $t_{max}$', pad=15) 
# ax2 = ax.secondary_xaxis('top')
# ax2.set_xlabel('No. Pauli Gadgets')
# ax2.tick_params(axis='x', color='black')
# ax2.set_xticks(ts, labels=[str(int(t/t_list[-1]*secdepth[-1])) for t in ts], minor=False)
# plt.legend()
# plt.show()

In [23]:
# fig, ax = plt.subplots()
# ts = [0]+t_list
# # ax.errorbar([0]+t_list,ump_spec_mean, xerr=extract_depth_at_t(t_list,mdepthp),yerr=ump_spec_std, label='physDrift',c='r')
# # ax.errorbar([0]+t_list,uqp_spec_mean, xerr=extract_depth_at_t(t_list,qdepthp),yerr=uqp_spec_std, label='qDrift',c='b')
# ax.errorbar([0]+t_list,np.mean(ump_spec,axis=0), xerr=extract_depth_at_t(t_list,mdepth),yerr=np.std(ump_spec,axis=0), label='physDrift',c='r')
# ax.errorbar([0]+t_list,np.mean(uqp_spec,axis=0), xerr=extract_depth_at_t(t_list,qdepth),yerr=np.std(uqp_spec,axis=0), label='qDrift',c='b')
# for i in range(len(ump_spec)):
#     ax.scatter([0]+t_list,ump_spec[i],marker='x',s=5,color='r')
# for i in range(len(uqp_spec)):
#     ax.scatter([0]+t_list,uqp_spec[i],marker='x',s=5,color='b')
# ax.set_xlabel(r'$t_{max}(s)$')
# ax.set_ylabel(r'Spectral error')
# ax.set_title(r'Spectral error over $t_{max}$ with permutation', pad=15) 
# ax2 = ax.secondary_xaxis('top')
# ax2.set_xlabel('No. Pauli Gadgets')
# ax2.tick_params(axis='x', color='black')
# ax2.set_xticks(ts, labels=[str(int(t/t_list[-1]*secdepth[-1])) for t in ts], minor=False)
# plt.legend()
# plt.show()

In [24]:
# fig, ax = plt.subplots()
# ts = [0]+t_list
# ax.errorbar([0]+t_list,ump_spec_ave, xerr=extract_depth_at_t(t_list,mdepth), label='physDrift',c='r')
# ax.errorbar([0]+t_list,uqp_spec_ave, xerr=extract_depth_at_t(t_list,qdepth),label='qDrift',c='b')
# ax.plot([0]+t_list,[np.abs(linalg.eig(u - u_exc)[0]).max() for u,u_exc in zip(u2p,uexc)],label='second protection',c='pink')
# ax.plot([0]+t_list,[np.abs(linalg.eig(u - u_exc)[0]).max() for u,u_exc in zip(u2,uexc)],label='second',c='purple')
# ax.plot([0]+t_list,[np.abs(linalg.eig(u - u_exc)[0]).max() for u,u_exc in zip(urp,uexc)],label='rand perm',c='black')
# ax.plot([0]+t_list,[np.abs(linalg.eig(u - u_exc)[0]).max() for u,u_exc in zip(utp,uexc)],label='lie protection',c='green')
# ax.plot([0]+t_list,[np.abs(linalg.eig(u - u_exc)[0]).max() for u,u_exc in zip(ut,uexc)],label='lie',c='yellow')
# ax.set_xlabel(r'$t_{max}(s)$')
# ax.set_ylabel('Spectral error')
# ax.set_title(r'Spectral error over $t_{max}$ comparison', pad=15) 
# ax2 = ax.secondary_xaxis('top')
# ax2.set_xlabel('No. Pauli Gadgets')
# ax2.tick_params(axis='x', color='black')
# ax2.set_xticks(ts, labels=[str(int(t/t_list[-1]*secdepth[-1])) for t in ts], minor=False)
# plt.legend()
# plt.show()